In [ ]:
import pandas as pd
import requests
import json
import csv
import os

In [ ]:
# making a POST request to the API endpoint
# q contains a search query for a specific collection of items.
# extracted data and stored in a list called 'all_collections'

api_url = 'https://search2.raritysniper.com/multi_search?use_cache=true&x-typesense-api-key=KEY'
q ={
    "searches": [
        {
            "query_by": "name",
            "sort_by": "launchDate:desc",
            "highlight_full_fields": "name",
            "collection": "collections",
            "q": "*",
            "facet_by": "blockchain,oneDayVolume,supply,thirtyDayVolume,totalVolume,sevenDayVolume,floorPrice",
            "max_facet_values": 20,
            "page": 1,
            "per_page": 250
        }
    ]
}
headers =  {"Content-Type":"application/json"}
all_collections = []
for i in range(1, 10):
    q['searches'][0]['page']=i
    response = requests.post(api_url, data=json.dumps(q), headers=headers)
    r = response.json()
    collections = r['results'][0]['hits']
    for c in collections:
        all_collections.append(c['document'])
        # print(c['document'])

In [ ]:
# create a DataFrame from the list of dictionaries

collection_df = pd.DataFrame(all_collections)
collection_df.to_csv('all_collections.csv', index=False)

In [ ]:
# add 'https://raritysniper.com/' before collection_name in 'collectionSlug' column
# add 'assets_' before collection_name in 'collectionSlug' column

collection_df['collectionURL'] = 'https://raritysniper.com/' + collection_df['collectionSlug']
collection_df['assetsname'] = 'assets_' + collection_df['collectionSlug']
collection_df

In [ ]:
# define fuction get_all_nfts
# iterate while function where hits > 0 
# create variable nft_keys, where there are only required keys [collectionId, CollectionName, nftId, rarityScore, rank]
# payload contains a search query for a specific collection of items.

def get_all_nfts(idx, name):
    collection_nfts = []
    page = 1
    payload = {
        "searches": [
            {
                "sort_by": "rank:asc,nftId:asc",
                "collection": name,
                "q": "*",
                "max_facet_values": 83,
                "page": page,
                "per_page": 250
            }
        ]
    }
    headers = {"Content-Type": "application/json"}
    r = requests.post(api_url, json=payload, headers=headers)
    while len(r.json()['results'][0]['hits']) > 0:
        nfts = r.json()['results'][0]['hits']
        for nft in nfts:
            nft_key = nft['document']['collectionId'], nft['document']['collectionName'], nft['document']['nftId'], nft['document']['rarityScore'] ,nft['document']['rank']
            collection_nfts.append(list(nft_key)) # extract values from dictionaries
        print(f'processed: {idx}, collection_name: {name}, page: {page}')
        page += 1
        payload["searches"][0]["page"] = page
        r = requests.post(api_url, json=payload, headers=headers)
    return collection_nfts


# define fucntion dum_csv
# create file csv file [collection_assets_batch_1.csv]
# take in a list of rows (nfts) and writes them to a CSV file named "collection_assets_batch_1.csv".

def dump_csv(nfts):
    file_exists = os.path.isfile('collection_assets_batch_1.csv')
    mode = 'a' if file_exists else 'w'
    with open('collection_assets_batch_1.csv', mode, newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists: # write headers if file doesn't exist
            writer.writerow(['collectionId', 'collectionName', 'nftId', 'rarityScore', 'rank'])
        for row in nfts:
            writer.writerow(row)

# code loops through the first 500 items of the 'assetsname' column in a DataFrame named 'collection_df'.
# create a empty list named [all_nft]
# returned nfts are appended to a list called all_nfts.

all_nfts=[]
for idx, asset_name in enumerate(collection_df['assetsname']):
    nfts = get_all_nfts(idx, asset_name)
    for nft in nfts:
        all_nfts.append(nft)
    if (idx + 1) % 10 == 0: # After every 10th iteration the dump_csv function is called.
        dump_csv(all_nfts) # Passing all_nfts as the argument
        all_nfts=[] # all_nfts is reset to an empty list
dump_csv(all_nfts)